# Back

## 0. Setting Environment

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.becchmark = True

## 1. Data Load

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, window_size):
        self.df = df
        self.window_size = window_size

    def __len__(self):
        return len(self.df) - self.window_size

    def __getitem__(self, idx):
        x = torch.tensor(self.df[idx:idx+self.window_size, :], dtype=torch.float)
        if self.df.shape[1] > 1:
            y = torch.tensor(self.df[idx+self.window_size, -1], dtype=torch.float)
        else:
            y = None
        return x, y

def create_data_loader(df, window_size, batch_size):
    dataset = TimeSeriesDataset(df, window_size)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

## 2. Data Preprocessing

## 3. Data Split (Train/Test)

## 4. Train

In [41]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out
    
    def train(self):
        for epoch in range(num_epochs):
            for i, (inputs, labels) in enumerate(train_loader):
                inputs = inputs.to(device)
                labels = labels.unsqueeze(1).to(device)

                # Forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 300 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                           .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
        return self
    
    def predict(self, x):
        return self.forward(x)

In [7]:
def set_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"current device: {device}")

In [8]:
def set_hyperparameter_A(input_size, hidden_size, num_layers, output_size):
    # 하이퍼파라미터
    input_size = input_size
    hidden_size = hidden_size  # 예측에 사용될 시간 윈도우 크기
    num_layers = num_layers
    output_size = output_size

In [9]:
def set_hyperparameter_B(num_epochs, window_size, batch_size, learning_rate):
    # 하이퍼파라미터
    num_epochs = num_epochs
    window_size = window_size  # 예측에 사용될 시간 윈도우 크기
    batch_size = batch_size
    learning_rate = learning_rate

In [10]:
def set_options():
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def normalization():
    # normalization
    scaler = MinMaxScaler()
    train_data = scaler.fit_transform(train_df.drop(['num_date_time', '건물번호', '일시'], axis=1).values)
    train_loader = create_data_loader(train_data, window_size, batch_size)

## 5. Test(evaluation)

## 6. Visualization

=====================================================

# User

## 0. Setting Environment

In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 

from tqdm.auto import tqdm

In [3]:
seed_everything(42)

## 1. Data Load

In [12]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

## 2. Data Preprocessing

In [13]:
# Train Data Preprocessing
train_df = train_df.drop(['일조(hr)','일사(MJ/m2)'], axis=1)

# 강수량 결측치 0.0으로 채우기
train_df['강수량(mm)'].fillna(0.0, inplace=True)

# 풍속, 습도 결측치 평균으로 채우고 반올림하기
train_df['풍속(m/s)'].fillna(round(train_df['풍속(m/s)'].mean(),2), inplace=True)
train_df['습도(%)'].fillna(round(train_df['습도(%)'].mean(),2), inplace=True)

train_df['month'] = train_df['일시'].apply(lambda x : float(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : float(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : float(x[9:11]))

# 순서 재배치
train_df = train_df[train_df.columns[:7].to_list() + train_df.columns[8:].to_list() + train_df.columns[7:8].to_list()]

## 3. Data Split (Train/Test)

## 4. Train

In [43]:
# set_hyperparameter_A / set_hyperparameter_B
# 하이퍼파라미터
input_size = 8  # feature의 개수
hidden_size = 64
num_layers = 2
output_size = 1
num_epochs = 1
window_size = 24  # 예측에 사용될 시간 윈도우 크기
batch_size = 128
learning_rate = 0.001

# set_device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"current device: {device}")

# normalization
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_df.drop(['num_date_time', '건물번호', '일시'], axis=1).values)
train_loader = create_data_loader(train_data, window_size, batch_size)

current device: cpu


In [44]:
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

In [45]:
# set_options
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [46]:
model.train()

Epoch [1/1], Step [300/1594], Loss: 0.0006
Epoch [1/1], Step [600/1594], Loss: 0.0041
Epoch [1/1], Step [900/1594], Loss: 0.0043
Epoch [1/1], Step [1200/1594], Loss: 0.0028
Epoch [1/1], Step [1500/1594], Loss: 0.0018


LSTM(
  (lstm): LSTM(8, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

## 5. Test (evaluation)

In [18]:
# Test Data Preprocessing
# 학습 데이터에서 마지막 행 가져오기
last_train_data = train_df.drop(['num_date_time', '건물번호', '일시',], axis=1).loc[204000-24:,:]

# 실수형 데이터로 변환
test_df['습도(%)'] = test_df['습도(%)'].astype('float64')

# 날짜 데이터 추가
test_df['month'] = test_df['일시'].apply(lambda x : float(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : float(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : float(x[9:11]))

# 전력소비량 열 생성
final_df = pd.concat((test_df.drop(['num_date_time', '건물번호', '일시',], axis=1), pd.DataFrame(np.zeros(test_df.shape[0]))),axis=1)
final_df = final_df.rename({0:'전력소비량(kWh)'},axis=1)

In [23]:
test_df = pd.concat((last_train_data, final_df)).reset_index(drop=True)
test_data = scaler.transform(test_df.values) # train과 동일하게 scaling
test_data.shape

(16824, 8)

In [32]:
test_loader = create_data_loader(test_data, window_size, 1)

In [47]:
x = torch.Tensor(test_data[i:i+window_size,:]).to(device)
x = x.view(1,window_size,-1)

model.predict(x)

tensor([[0.0215]], grad_fn=<AddmmBackward0>)

In [39]:
model.eval()

TypeError: train() takes 1 positional argument but 2 were given

In [48]:
test_predictions = []

with torch.no_grad():
    for i in range(test_data.shape[0] - window_size):
        x = torch.Tensor(test_data[i:i+window_size,:]).to(device)
        new_x = model(x.view(1,window_size,-1))
        
        test_data[i+window_size,-1] = new_x # 입력 업데이트
        test_predictions.append(new_x.detach().cpu().numpy().item()) # 예측 결과 저장

In [ ]:
predictions = scaler.inverse_transform(test_data)[24:,-1] # 원래 scale로 복구

sample_submission['answer'] = predictions

sample_submission.to_csv('lstm_baseline_submission.csv', index=False)

## 6. Visualization